In [13]:
import pandas as pd

# BUVATN VARIERENDE STRØMPRIS

# 1. Last inn strømpriser
df = pd.read_csv("strom2024.csv", sep=";")
df.columns = df.columns.str.strip()
pris_kolonne = df.columns[1]
df[pris_kolonne] = df[pris_kolonne].astype(str).str.replace(",", ".", regex=False).astype(float)

df["Tid"] = pd.to_datetime(
    df["Dato/klokkeslett"].str.replace(r" Kl\. (\d{2})-\d{2}", r" \1:00", regex=True),
    format="%Y-%m-%d %H:%M", errors="coerce"
)
df = df.dropna(subset=["Tid"])
df = df.set_index("Tid")
df["Måned"] = df.index.month

# 2. Konstanter
total_tilsig_m3 = 6_477_921             # tilgjengelig vann (m³)
energi_per_m3_pumpe = 0.138260917       # kWh per m³ for pumping
energi_per_m3_turbin = 1.157            # kWh per m³ for turbin
slukeevne_turbin = 14_760               # m³ per time turbin
slukeevne_pumpe = 4320                  # m³ per time pumpe
timer_pumpe_sommer = 6.0
timer_pumpe_vinter = 2.7

timer_sommer = int(timer_pumpe_sommer * 153)
timer_vinter = int(timer_pumpe_vinter * 212)

# 3. Fordel tilsig mellom sommer og vinter (direkte gitt her)
tilsig_sommer = 3_999_152
tilsig_vinter = 2_478_769

# 4. Finn billigste pumpetimer separat for sommer og vinter
sommer_df = df[df["Måned"].isin([5, 6, 7, 8, 9])]
billigste_sommer = sommer_df.nsmallest(timer_sommer, pris_kolonne)

vinter_df = df[~df["Måned"].isin([5, 6, 7, 8, 9])]
billigste_vinter = vinter_df.nsmallest(timer_vinter, pris_kolonne)

# Finn de 5 billigste pumpetimene hver for sommer og vinter (for oversikt)
billigste_5_sommer = billigste_sommer.nsmallest(5, pris_kolonne)
billigste_5_vinter = billigste_vinter.nsmallest(5, pris_kolonne)

# Finn de dyreste pumpetimene for sommer og vinter
dyreste_pumpe_sommer = billigste_sommer.nlargest(5, pris_kolonne)
dyreste_pumpe_vinter = billigste_vinter.nlargest(5, pris_kolonne)

# 5. Beregn pumpekostnader
kost_sommer = (billigste_sommer[pris_kolonne] * energi_per_m3_pumpe * slukeevne_pumpe).sum()
kost_vinter = (billigste_vinter[pris_kolonne] * energi_per_m3_pumpe * slukeevne_pumpe).sum()
total_kostnad = kost_sommer + kost_vinter

# 6. Produksjonsinntekter – bruk dyreste timer i året
total_timer_produksjon = total_tilsig_m3 / slukeevne_turbin
dyreste_timer = df[pris_kolonne].nlargest(int(total_timer_produksjon))
produksjon_inntekt = (dyreste_timer * energi_per_m3_turbin * slukeevne_turbin).sum()

# 7. Resultat
fortjeneste = produksjon_inntekt - total_kostnad

# 8. Regne ut sum av fremtidige inntekter
r = 0.06
n = 30
diskonteringsfaktor = (1 - (1 + r)**(-n))/r
sum_inntekter = fortjeneste * diskonteringsfaktor

# 9. Print ut resultater
print(f"Kostnad pumping:     {round(total_kostnad, 2):,} kr")
print(f"Inntekt produksjon:  {round(produksjon_inntekt, 2):,} kr")
print(f"Fortjeneste totalt:  {round(fortjeneste, 2):,} kr")
print(f"Sum av fremtidige inntekter:  {round(sum_inntekter, 2):,} kr")

print(f"\nAntall timer pumpet (sommer): {timer_sommer}")
print(f"Antall timer pumpet (vinter): {timer_vinter}")
print(f"Antall produksjonstimer: {round(total_timer_produksjon)}")

print("\n=== 5 billigste pumpetimer sommer ===")
print(billigste_5_sommer[[pris_kolonne]])

print("\n=== 5 billigste pumpetimer vinter ===")
print(billigste_5_vinter[[pris_kolonne]])

print("\n=== 5 dyreste pumpetimer sommer ===")
print(dyreste_pumpe_sommer[[pris_kolonne]])

print("\n=== 5 dyreste pumpetimer vinter ===")
print(dyreste_pumpe_vinter[[pris_kolonne]])

print("\n=== 5 dyreste produksjonstimer ===")
print(df.loc[dyreste_timer.index].nlargest(5, pris_kolonne)[[pris_kolonne]])

print("\n=== 5 billigste produksjonstimer ===")
print(df.loc[dyreste_timer.index].nsmallest(5, pris_kolonne)[[pris_kolonne]])



Kostnad pumping:     32,677.17 kr
Inntekt produksjon:  6,201,519.8 kr
Fortjeneste totalt:  6,168,842.64 kr
Sum av fremtidige inntekter:  84,913,077.29 kr

Antall timer pumpet (sommer): 918
Antall timer pumpet (vinter): 572
Antall produksjonstimer: 439

=== 5 billigste pumpetimer sommer ===
                         NO4
Tid                         
2024-08-24 13:00:00 -0.03526
2024-08-25 14:00:00 -0.03272
2024-08-24 14:00:00 -0.03078
2024-08-24 12:00:00 -0.03007
2024-08-25 15:00:00 -0.02850

=== 5 billigste pumpetimer vinter ===
                         NO4
Tid                         
2024-11-13 18:00:00 -0.02359
2024-10-30 03:00:00 -0.01565
2024-10-30 02:00:00 -0.01494
2024-10-30 00:00:00 -0.01387
2024-10-30 01:00:00 -0.01316

=== 5 dyreste pumpetimer sommer ===
                         NO4
Tid                         
2024-09-13 08:00:00  0.11678
2024-06-02 11:00:00  0.11662
2024-08-19 22:00:00  0.11661
2024-05-16 16:00:00  0.11653
2024-06-09 10:00:00  0.11640

=== 5 dyreste pumpetime

In [14]:
# DJUPÅVATNET VARIERENDE STRØMPRIS

# 1. Last inn strømpriser
df = pd.read_csv("strom2024.csv", sep=";")
df.columns = df.columns.str.strip()
pris_kolonne = df.columns[1]
df[pris_kolonne] = df[pris_kolonne].astype(str).str.replace(",", ".", regex=False).astype(float)

df["Tid"] = pd.to_datetime(
    df["Dato/klokkeslett"].str.replace(r" Kl\. (\d{2})-\d{2}", r" \1:00", regex=True),
    format="%Y-%m-%d %H:%M", errors="coerce"
)
df = df.dropna(subset=["Tid"])
df = df.set_index("Tid")
df["Måned"] = df.index.month

# 2. Konstanter
total_tilsig_m3 = 3_163_209             # tilgjengelig vann (m³)
energi_per_m3_pumpe = 0.199005511      # kWh per m³ for pumping
energi_per_m3_turbin = 1.157            # kWh per m³ for turbin
slukeevne_turbin = 14_760               # m³ per time turbin
slukeevne_pumpe = 2160                  # m³ per time pumpe
timer_pumpe_sommer = 6.0
timer_pumpe_vinter = 2.7

timer_sommer = int(timer_pumpe_sommer * 153)
timer_vinter = int(timer_pumpe_vinter * 212)

# 3. Fordel tilsig mellom sommer og vinter (direkte gitt her)
tilsig_sommer = 1_951_273
tilsig_vinter = 1_211_937

# 4. Finn de billigste pumpetimene
sommer_df = df[df["Måned"].isin([5, 6, 7, 8, 9])]
billigste_sommer = sommer_df.nsmallest(timer_sommer, pris_kolonne)

vinter_df = df[~df["Måned"].isin([5, 6, 7, 8, 9])]
billigste_vinter = vinter_df.nsmallest(timer_vinter, pris_kolonne)

# 5. Beregn pumpekostnader
kost_sommer = (billigste_sommer[pris_kolonne] * energi_per_m3_pumpe * slukeevne_pumpe).sum()
kost_vinter = (billigste_vinter[pris_kolonne] * energi_per_m3_pumpe * slukeevne_pumpe).sum()
total_kostnad = kost_sommer + kost_vinter
# 6. Produksjonsinntekter – bruk dyreste timer i året
total_timer_produksjon = total_tilsig_m3 / slukeevne_turbin
dyreste_timer = df[pris_kolonne].nlargest(int(total_timer_produksjon))
produksjon_inntekt = (dyreste_timer * energi_per_m3_turbin * slukeevne_turbin).sum()

# 7. Resultat
fortjeneste = produksjon_inntekt - total_kostnad

# 8. Regne ut sum av fremtidige inntekter

r = 0.06
n = 30

diskonteringsfaktor = (1 - (1 + r)**(-n))/r


sum_inntekter = fortjeneste * diskonteringsfaktor


# 9. Print ut resultater
print(f"Kostnad pumping:     {round(total_kostnad, 2):,} kr")
print(f"Inntekt produksjon:  {round(produksjon_inntekt, 2):,} kr")
print(f"Fortjeneste totalt:  {round(fortjeneste, 2):,} kr")
print(f"Sum av fremtidige inntekter:  {round(sum_inntekter, 2):,} kr")

print(f"\nAntall timer pumpet (sommer): {timer_sommer}")
print(f"Antall timer pumpet (vinter): {timer_vinter}")
print(f"Antall produksjonstimer: {round(total_timer_produksjon)}")

print("\n=== 5 billigste pumpetimer sommer ===")
print(billigste_5_sommer[[pris_kolonne]])

print("\n=== 5 billigste pumpetimer vinter ===")
print(billigste_5_vinter[[pris_kolonne]])

print("\n=== 5 dyreste pumpetimer sommer ===")
print(dyreste_pumpe_sommer[[pris_kolonne]])

print("\n=== 5 dyreste pumpetimer vinter ===")
print(dyreste_pumpe_vinter[[pris_kolonne]])

print("\n=== 5 dyreste produksjonstimer ===")
print(df.loc[dyreste_timer.index].nlargest(5, pris_kolonne)[[pris_kolonne]])

print("\n=== 5 billigste produksjonstimer  ===")
print(df.loc[dyreste_timer.index].nsmallest(5, pris_kolonne)[[pris_kolonne]])


Kostnad pumping:     23,516.9 kr
Inntekt produksjon:  3,429,384.41 kr
Fortjeneste totalt:  3,405,867.51 kr
Sum av fremtidige inntekter:  46,881,191.14 kr

Antall timer pumpet (sommer): 918
Antall timer pumpet (vinter): 572
Antall produksjonstimer: 214

=== 5 billigste pumpetimer sommer ===
                         NO4
Tid                         
2024-08-24 13:00:00 -0.03526
2024-08-25 14:00:00 -0.03272
2024-08-24 14:00:00 -0.03078
2024-08-24 12:00:00 -0.03007
2024-08-25 15:00:00 -0.02850

=== 5 billigste pumpetimer vinter ===
                         NO4
Tid                         
2024-11-13 18:00:00 -0.02359
2024-10-30 03:00:00 -0.01565
2024-10-30 02:00:00 -0.01494
2024-10-30 00:00:00 -0.01387
2024-10-30 01:00:00 -0.01316

=== 5 dyreste pumpetimer sommer ===
                         NO4
Tid                         
2024-09-13 08:00:00  0.11678
2024-06-02 11:00:00  0.11662
2024-08-19 22:00:00  0.11661
2024-05-16 16:00:00  0.11653
2024-06-09 10:00:00  0.11640

=== 5 dyreste pumpetime

In [15]:
# BUVATNET FLAT STRØMPRIS

# Konstanter
strømpris = 0.27005
total_tilsig_m3 = 6_477_921              # tilgjengelig vann (m³)
energi_per_m3_pumpe = 0.138260917        # kWh per m³ for pumping
energi_per_m3_turbin = 1.157             # kWh per m³ for turbin
slukeevne_turbin = 14_760                # m³ per time turbin
slukeevne_pumpe = 4320                   # m³ per time pumpe
timer_pumpe_sommer = 6.0
timer_pumpe_vinter = 2.7
sommer = 153
vinter = 212

# Beregn pumpekostnader
kost_sommer = timer_pumpe_sommer * sommer * strømpris * energi_per_m3_pumpe * slukeevne_pumpe
kost_vinter = timer_pumpe_vinter * vinter * strømpris * energi_per_m3_pumpe * slukeevne_pumpe
total_kostnad = kost_sommer + kost_vinter

# Produksjonsinntekter – bruk dyreste timer i året
total_timer_produksjon = total_tilsig_m3 / slukeevne_turbin
produksjon_inntekt = total_timer_produksjon * strømpris * energi_per_m3_turbin * slukeevne_turbin

# Resultat
fortjeneste = produksjon_inntekt - total_kostnad

# Regne ut sum av fremtidige inntekter
r = 0.06  # kalkulasjonsrente
n = 30    # antall år

diskonteringsfaktor = (1 - (1 + r)**(-n))/r
sum_inntekter = fortjeneste * diskonteringsfaktor

# Print ut resultater
print(f"Kostnad pumping:                {round(total_kostnad, 2):,} kr")
print(f"Inntekt produksjon:             {round(produksjon_inntekt, 2):,} kr")
print(f"Fortjeneste totalt:             {round(fortjeneste, 2):,} kr")
print(f"Sum av fremtidige inntekter:    {round(sum_inntekter, 2):,} kr")

Kostnad pumping:                240,397.64 kr
Inntekt produksjon:             2,024,012.49 kr
Fortjeneste totalt:             1,783,614.85 kr
Sum av fremtidige inntekter:    24,551,157.21 kr


In [16]:
# DJUPÅVATN FLAT STRØMPRIS

# Konstanter
strømpris = 0.27005
total_tilsig_m3 = 3_163_209               # tilgjengelig vann (m³)
energi_per_m3_pumpe = 0.199005511        # kWh per m³ for pumping
energi_per_m3_turbin = 1.157             # kWh per m³ for turbin
slukeevne_turbin = 14_760                # m³ per time turbin
slukeevne_pumpe = 2160                   # m³ per time pumpe
timer_pumpe_sommer = 6.0
timer_pumpe_vinter = 2.7
sommer = 153
vinter = 212

# Beregn pumpekostnader
kost_sommer = timer_pumpe_sommer * sommer * strømpris * energi_per_m3_pumpe * slukeevne_pumpe
kost_vinter = timer_pumpe_vinter * vinter * strømpris * energi_per_m3_pumpe * slukeevne_pumpe
total_kostnad = kost_sommer + kost_vinter

# Produksjonsinntekter – bruk dyreste timer i året
total_timer_produksjon = total_tilsig_m3 / slukeevne_turbin
produksjon_inntekt = total_timer_produksjon * strømpris * energi_per_m3_turbin * slukeevne_turbin

# Resultat
fortjeneste = produksjon_inntekt - total_kostnad

# Regne ut sum av fremtidige inntekter
r = 0.06  # kalkulasjonsrente
n = 30    # antall år

diskonteringsfaktor = (1 - (1 + r)**(-n))/r
sum_inntekter = fortjeneste * diskonteringsfaktor

print(diskonteringsfaktor)

# Print ut resultater
print(f"Kostnad pumping:                {round(total_kostnad, 2):,} kr")
print(f"Inntekt produksjon:             {round(produksjon_inntekt, 2):,} kr")
print(f"Fortjeneste totalt:             {round(fortjeneste, 2):,} kr")
print(f"Sum av fremtidige inntekter:    {round(sum_inntekter, 2):,} kr")


13.764831151489428
Kostnad pumping:                173,007.88 kr
Inntekt produksjon:             988,337.85 kr
Fortjeneste totalt:             815,329.97 kr
Sum av fremtidige inntekter:    11,222,879.42 kr
